In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
df_programs = []

for i in range(4):
    #Train data
    df = pd.read_excel('BSA-dataset_2122_2223-Train_stud_a.xlsx', sheet_name=i)
    df1 = df[df["Year"] == "21/22"]
    df2 =df[df["Year"] == "22/23"]

    df1_val = df1.sample(frac=0.3, random_state=25)
    df1_train = df1.drop(df1_val.index)

    df1_val["Origin"] = "test"
    df1_train["Origin"] = "train"

    df1_final = pd.concat([df1_val,df1_train])



    df2_val = df2.sample(frac=0.3, random_state=25)
    df2_train = df2.drop(df2_val.index)

    df2_val["Origin"] = "test"
    df2_train["Origin"] = "train"


    df2_final = pd.concat([df1_final,df2_train]) 


    df_final = pd.concat([df2_val,df2_final]) 
    df_programs.append(df_final)


In [ ]:
df = pd.concat(df_programs,ignore_index=True)
# columns_to_drop = [col for col in df.columns if col.startswith("Co")]
# df = df.drop(columns=columns_to_drop)
df.columns

In [ ]:
import itertools


In [ ]:
import pandas as pd
import numpy as np
import itertools
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def Plot(df):
    # Define the passing grade
    passing_grade = 5.5

    # Define the courses for each program
    courses = {
        'Program1': ['Course3', 'Course9', 'Course7-1', 'Course8-1'],
        'Program2': ['Course1', 'Course11', 'Course2-1', 'Course10-1'],
        'Program3': ['Course1', 'Course12', 'Course2-1', 'Course10-1'],
        'Program4': ['Course3', 'Course4', 'Course6-1', 'Course5-1']
    }

    # Create pass/fail indicators and scenarios
    df['Scenario'] = ''
    for program, course_list in courses.items():
        for course in course_list:
            df[f'{course}_pass'] = (df[course] >= passing_grade).astype(int)
        df.loc[df['Program'] == program, 'Scenario'] = df[df['Program'] == program][[f'{course}_pass' for course in course_list]].astype(str).agg(''.join, axis=1)

    # Define custom order of scenarios
    scenarios = sorted([''.join(seq) for seq in itertools.product('01', repeat=4)], key=lambda x: (x.count('1'), x))

    # Define the order of 'Program' facet columns and BSA categories
    program_order = ['Program1', 'Program2', 'Program3', 'Program4']
    bsa_order = ['STGA', 'NE', 'PS', 'DI']
    bsa_colors = {
        'STGA': '#4d4d4d',
        'NE': '#b30000',
        'PS': '#258e25',
        'DI': 'lightblue'
    }

    # Prepare data for plotting
    df_agg = df.groupby(['Year', 'Program', 'BSA', 'Scenario']).size().reset_index(name='Count')

    # Create a DataFrame with all possible combinations of Year, Program, BSA, and Scenario
    all_combinations = pd.MultiIndex.from_product(
        [df['Year'].unique(), program_order, bsa_order, scenarios],
        names=['Year', 'Program', 'BSA', 'Scenario']
    ).to_frame(index=False)

    # Merge with all possible combinations to include missing combinations
    df_full = pd.merge(all_combinations, df_agg, on=['Year', 'Program', 'BSA', 'Scenario'], how='left')

    # Fill missing counts with 0
    df_full['Count'] = df_full['Count'].fillna(0)

    # Create a figure with subplots arranged in a 8x1 grid
    fig = make_subplots(rows=8, cols=1, subplot_titles=[f'{program} {year}' for program in program_order for year in df['Year'].unique()], vertical_spacing=0.04)

    # Loop through each 'Program' facet column and 'Year' facet row
    row_counter = 1
    for program in program_order:
        for year in df['Year'].unique():
            # Filter data for the current program and year
            df_subset = df_full[(df_full['Program'] == program) & (df_full['Year'] == year)]
            
            # Loop through each 'BSA' category
            for bsa in bsa_order:
                # Filter data for the current BSA category
                df_bsa = df_subset[df_subset['BSA'] == bsa]
                
                # Create bar trace
                bar = go.Bar(
                    x=pd.Categorical(df_bsa['Scenario'], categories=scenarios, ordered=True),
                    y=df_bsa['Count'],
                    name=bsa,
                    marker=dict(color=bsa_colors[bsa]),
                    text=df_bsa['Count'],  # Add text on top of each bar
                    textposition='auto'  # Automatically position the text
                )
                
                # Add trace to the subplot
                fig.add_trace(bar, row=row_counter, col=1)
            
            row_counter += 1

    # Update layout
    fig.update_layout(title='Count of Scenarios Layered by BSA and Program', height=2400, width=800)
    fig.update_layout(bargap=0.2)
    fig.update_xaxes(title_text='Scenario')
    fig.update_yaxes(title_text='Count')
    fig.update_layout(showlegend=False)  # Hide the legend
    fig.update_annotations(font_size=12)  # Update font size for subplot titles
    fig.update_layout(height=2200, width=1200, bargap=0.2)
    fig.show()


In [ ]:
df_try = df
Plot(df_try)

In [ ]:
df

In [ ]:
import pandas as pd
import numpy as np
import itertools
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot2(df):
    # Define the passing grade
    passing_grade = 5.5

    # Define the courses for each program
    courses = {
        'Program1': ['Course3', 'Course9', 'Course7-1', 'Course8-1'],
        'Program2': ['Course1', 'Course11', 'Course2-1', 'Course10-1'],
        'Program3': ['Course1', 'Course12', 'Course2-1', 'Course10-1'],
        'Program4': ['Course3', 'Course4', 'Course6-1', 'Course5-1']
    }

    # Create pass/fail indicators and scenarios
    df['Scenario'] = ''
    for program, course_list in courses.items():
        for course in course_list:
            df[f'{course}_pass'] = (df[course] >= passing_grade).astype(int)
        df.loc[df['Program'] == program, 'Scenario'] = df[df['Program'] == program][[f'{course}_pass' for course in course_list]].astype(str).agg(''.join, axis=1)

    # Define custom order of scenarios
    scenarios = [1,0,-1]

    # Define the order of 'Program' facet columns and BSA categories
    program_order = ['Program1', 'Program2', 'Program3', 'Program4']
    bsa_order = ['STGA', 'NE', 'PS', 'DI']
    bsa_colors = {
        'STGA': '#4d4d4d',
        'NE': '#b30000',
        'PS': '#258e25',
        'DI': 'lightblue'
    }



    # Create a figure with subplots arranged in a 8x1 grid
    fig = make_subplots(rows=8, cols=4, subplot_titles=[f'{program} {year} {course}' for program in program_order for year in df['Year'].unique() for course in courses[program]], vertical_spacing=0.04)

    # Loop through each 'Program' facet column and 'Year' facet row
    row_counter = 1
    for program in program_order:
        for year in df['Year'].unique():
            # Filter data for the current program and year
            df_subset = df[(df['Program'] == program) & (df['Year'] == year)]
            
            for i, course in enumerate(courses[program], start=1):
                    # Prepare data for plotting
                df_subset2 = df_subset.groupby(['BSA', f'{course}_pass']).size().reset_index(name='Count')
                # Loop through each 'BSA' category
                for bsa in bsa_order:
                    # Filter data for the current BSA category
                    df_bsa = df_subset2[df_subset2['BSA'] == bsa]
                    
                    # Create bar trace
                    bar = go.Bar(
                        x=pd.Categorical(df_bsa[f'{course}_pass'], categories=scenarios, ordered=True),
                        y=df_bsa['Count'],
                        name=bsa,
                        marker=dict(color=bsa_colors[bsa]),
                        text=df_bsa['Count'],  # Add text on top of each bar
                        textposition='auto'  # Automatically position the text
                    )
                    
                    # Add trace to the subplot
                    fig.add_trace(bar, row=row_counter, col=i)
            
            row_counter += 1

    # Update layout
    fig.update_layout(title='Count of Scenarios Layered by BSA and Program', height=2400, width=800)
    fig.update_layout(bargap=0.2)
    #fig.update_xaxes(title_text='Scenario')
    fig.update_yaxes(title_text='Count')
    fig.update_layout(showlegend=False)  # Hide the legend
    fig.update_annotations(font_size=12)  # Update font size for subplot titles
    fig.update_layout(height=2200, width=1200, bargap=0.2)
    fig.show()


plot2(df)